# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 20 2023 11:47AM,261506,19,MSP220985,"Methapharm, Inc.",Released
1,Apr 20 2023 11:35AM,261505,21,854367,"NBTY Global, Inc.",Released
2,Apr 20 2023 11:23AM,261503,15,VT80186371,"Virtus Pharmaceuticals, LLC",Released
3,Apr 20 2023 11:22AM,261501,15,ALIL513500-01,"Alliance Pharma, Inc.",Released
4,Apr 20 2023 11:22AM,261501,15,30015871,"Alliance Pharma, Inc.",Released
5,Apr 20 2023 11:22AM,261501,15,30015872,"Alliance Pharma, Inc.",Released
6,Apr 20 2023 11:22AM,261502,10,Enova-11693,Emerginnova,Released
7,Apr 20 2023 11:17AM,261460,15,PNC442792A,"Person & Covey, Inc.",Released
8,Apr 20 2023 11:17AM,261482,15,PNC442827B,"Person & Covey, Inc.",Released
9,Apr 20 2023 11:16AM,261483,15,9387621,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,261501,Released,3
30,261502,Released,1
31,261503,Released,1
32,261505,Released,1
33,261506,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261501,NaN,NaN,3.0
261502,NaN,NaN,1.0
261503,NaN,NaN,1.0
261505,NaN,NaN,1.0
261506,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261400,0.0,37.0,5.0
261432,0.0,0.0,1.0
261448,0.0,0.0,5.0
261452,0.0,0.0,1.0
261460,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261400,0,37,5
261432,0,0,1
261448,0,0,5
261452,0,0,1
261460,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261400,0,37,5
1,261432,0,0,1
2,261448,0,0,5
3,261452,0,0,1
4,261460,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261400,,37,5
1,261432,,,1
2,261448,,,5
3,261452,,,1
4,261460,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 20 2023 11:47AM,261506,19,"Methapharm, Inc."
1,Apr 20 2023 11:35AM,261505,21,"NBTY Global, Inc."
2,Apr 20 2023 11:23AM,261503,15,"Virtus Pharmaceuticals, LLC"
3,Apr 20 2023 11:22AM,261501,15,"Alliance Pharma, Inc."
6,Apr 20 2023 11:22AM,261502,10,Emerginnova
7,Apr 20 2023 11:17AM,261460,15,"Person & Covey, Inc."
8,Apr 20 2023 11:17AM,261482,15,"Person & Covey, Inc."
9,Apr 20 2023 11:16AM,261483,15,"Brookfield Pharmaceuticals, LLC"
10,Apr 20 2023 11:16AM,261500,21,"NBTY Global, Inc."
11,Apr 20 2023 11:07AM,261499,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 20 2023 11:47AM,261506,19,"Methapharm, Inc.",,,1
1,Apr 20 2023 11:35AM,261505,21,"NBTY Global, Inc.",,,1
2,Apr 20 2023 11:23AM,261503,15,"Virtus Pharmaceuticals, LLC",,,1
3,Apr 20 2023 11:22AM,261501,15,"Alliance Pharma, Inc.",,,3
4,Apr 20 2023 11:22AM,261502,10,Emerginnova,,,1
5,Apr 20 2023 11:17AM,261460,15,"Person & Covey, Inc.",,,1
6,Apr 20 2023 11:17AM,261482,15,"Person & Covey, Inc.",,,1
7,Apr 20 2023 11:16AM,261483,15,"Brookfield Pharmaceuticals, LLC",,,1
8,Apr 20 2023 11:16AM,261500,21,"NBTY Global, Inc.",,1,
9,Apr 20 2023 11:07AM,261499,10,Emerginnova,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 11:47AM,261506,19,"Methapharm, Inc.",1,,
1,Apr 20 2023 11:35AM,261505,21,"NBTY Global, Inc.",1,,
2,Apr 20 2023 11:23AM,261503,15,"Virtus Pharmaceuticals, LLC",1,,
3,Apr 20 2023 11:22AM,261501,15,"Alliance Pharma, Inc.",3,,
4,Apr 20 2023 11:22AM,261502,10,Emerginnova,1,,
5,Apr 20 2023 11:17AM,261460,15,"Person & Covey, Inc.",1,,
6,Apr 20 2023 11:17AM,261482,15,"Person & Covey, Inc.",1,,
7,Apr 20 2023 11:16AM,261483,15,"Brookfield Pharmaceuticals, LLC",1,,
8,Apr 20 2023 11:16AM,261500,21,"NBTY Global, Inc.",,1,
9,Apr 20 2023 11:07AM,261499,10,Emerginnova,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 11:47AM,261506,19,"Methapharm, Inc.",1,,
1,Apr 20 2023 11:35AM,261505,21,"NBTY Global, Inc.",1,,
2,Apr 20 2023 11:23AM,261503,15,"Virtus Pharmaceuticals, LLC",1,,
3,Apr 20 2023 11:22AM,261501,15,"Alliance Pharma, Inc.",3,,
4,Apr 20 2023 11:22AM,261502,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 11:47AM,261506,19,"Methapharm, Inc.",1.0,NaN,NaN
1,Apr 20 2023 11:35AM,261505,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Apr 20 2023 11:23AM,261503,15,"Virtus Pharmaceuticals, LLC",1.0,NaN,NaN
3,Apr 20 2023 11:22AM,261501,15,"Alliance Pharma, Inc.",3.0,NaN,NaN
4,Apr 20 2023 11:22AM,261502,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 11:47AM,261506,19,"Methapharm, Inc.",1.0,0.0,0.0
1,Apr 20 2023 11:35AM,261505,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Apr 20 2023 11:23AM,261503,15,"Virtus Pharmaceuticals, LLC",1.0,0.0,0.0
3,Apr 20 2023 11:22AM,261501,15,"Alliance Pharma, Inc.",3.0,0.0,0.0
4,Apr 20 2023 11:22AM,261502,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1568952,8.0,10.0,0.0
15,1830277,17.0,37.0,0.0
19,3399160,89.0,16.0,1.0
21,784491,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1568952,8.0,10.0,0.0
1,15,1830277,17.0,37.0,0.0
2,19,3399160,89.0,16.0,1.0
3,21,784491,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,8.0,10.0,0.0
1,15,17.0,37.0,0.0
2,19,89.0,16.0,1.0
3,21,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,8.0
1,15,Released,17.0
2,19,Released,89.0
3,21,Released,1.0
4,10,Executing,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,21
Status,,,,
Completed,0.0,0.0,1.0,0.0
Executing,10.0,37.0,16.0,2.0
Released,8.0,17.0,89.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,21
0,Completed,0.0,0.0,1.0,0.0
1,Executing,10.0,37.0,16.0,2.0
2,Released,8.0,17.0,89.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,21
0,Completed,0.0,0.0,1.0,0.0
1,Executing,10.0,37.0,16.0,2.0
2,Released,8.0,17.0,89.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()